In [1]:
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

In [8]:
from transformers import pipeline

In [3]:
# Load three datasets: dataset contains all the data without labels, dataset_train_eval contains data with labels
# test contains data without labels
from datasets import load_dataset
from datasets import DatasetDict
dataset = load_dataset("csv",  data_files="../indexing/output.csv")
# We split the data test:eval 70:30 
dataset_train_eval = (load_dataset("csv", data_files="./balanced_labels_posneg.csv", split = "train").train_test_split(test_size=0.3))
dataset_test_eval = dataset_train_eval["test"].train_test_split(test_size=0.5)
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_eval['train'],
    'test': dataset_test_eval['test'],
    'valid': dataset_test_eval['train']})
test = load_dataset("csv", data_files="./test.csv")
# dataset["train"][100]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-df29f072a3b1628c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-61716cd1da63ae18/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-ec53b3ecffeb1dbe/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# Rename the column label to label_fine_grained [0,5]
# dataset_train_eval = dataset_train_eval.rename_column("label", "label_fine_grained")

In [4]:
# Rename the column label_posneg to label [0, 2]
# dataset_train_eval = dataset_train_eval.rename_column("sentiment", "label")
train_test_valid_dataset = train_test_valid_dataset.rename_column("sentiment", "label")

In [5]:
# Check the dataset_train_eval
train_test_valid_dataset["train"]

Dataset({
    features: ['Unnamed: 0', 'TEXT', 'label'],
    num_rows: 700
})

In [6]:
test

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'DATE', 'AUTHOR', 'TEXT', 'FAVORITE', 'year', 'manufacturer', 'model', 'Initial Index'],
        num_rows: 39347
    })
})

In [13]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer


2023-04-04 21:02:58.011486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Tokenizer: Hello, How are You: [Hello, How, Are, You]
# Specify Max Length 42 for now, the default model does not have a predefined maximum length

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["TEXT"], padding="max_length", truncation=True, max_length=142)


tokenized_datasets_train = train_test_valid_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_labelled_test = train_test_valid_dataset['test'].map(tokenize_function, batched=True)
tokenized_datasets_val = train_test_valid_dataset['test'].map(tokenize_function, batched=True)
tokenized_dataset_test = test['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-61716cd1da63ae18/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0203b19236f884a7.arrow
Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-ec53b3ecffeb1dbe/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-bab8b8f122e9eeee.arrow


In [9]:
# We load the model, and specify the num_labels as 2 [pos, neg]
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2,ignore_mismatched_sizes=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [10]:
# Evaluation metrics, we can include more such as: F-measure
import numpy as np
import evaluate

2023-03-27 21:34:33.919111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# import pytorch and check for GPU availability, for using torch.argmax in the future
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cpu


In [27]:
import evaluate

metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [28]:
# Convert predictions to logits and use argmax function, need to change np.argmax to torch.argmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.from_numpy(logits)
    labels = torch.from_numpy(labels)
    predictions = torch.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1-score": f1}

In [29]:
# Hyperparameter training, for now using default hyperparameters


training_args = TrainingArguments(output_dir="test_trainer", logging_steps = 88, evaluation_strategy="epoch")

In [30]:
# We can use optuna to do hyperparameter search
# def optuna_hp_space(trial):
#    return {
#        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
#    }

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.116800,0.464816,0.940000,0.900000,1.000000
2,0.083200,0.328884,0.940000,0.909091,0.987654
3,0.000900,0.253719,0.960000,0.974684,0.950617


TrainOutput(global_step=264, training_loss=0.06694542924224427, metrics={'train_runtime': 1247.7633, 'train_samples_per_second': 1.683, 'train_steps_per_second': 0.212, 'total_flos': 153241634196000.0, 'train_loss': 0.06694542924224427, 'epoch': 3.0})

In [33]:
predictions = trainer.predict(tokenized_datasets_labelled_test)


In [34]:
trainer.evaluate()

{'eval_loss': 0.2537185847759247,
 'eval_accuracy': 0.96,
 'eval_precision': 0.9746835443037974,
 'eval_recall': 0.9506172839506173,
 'eval_runtime': 20.2551,
 'eval_samples_per_second': 7.406,
 'eval_steps_per_second': 0.938,
 'epoch': 3.0}

In [35]:
print(predictions.predictions.shape)

(150, 2)


In [36]:
preds = torch.argmax(torch.from_numpy(predictions.predictions), axis=-1)

In [37]:
for x in range(10):
    print("Label: ", predictions.label_ids[x], "Prediction: ", preds[x])

Label:  0 Prediction:  tensor(0)
Label:  1 Prediction:  tensor(1)
Label:  0 Prediction:  tensor(0)
Label:  0 Prediction:  tensor(0)
Label:  1 Prediction:  tensor(1)
Label:  1 Prediction:  tensor(1)
Label:  0 Prediction:  tensor(0)
Label:  1 Prediction:  tensor(1)
Label:  0 Prediction:  tensor(0)
Label:  0 Prediction:  tensor(0)


In [38]:
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.96, 'f1': 0.9625}

In [40]:
trainer.save_model("trained_model")

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("./trained_model/", num_labels=2,ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, max_length=142, padding = 'max_length', truncation=True, batch_size=16)

In [15]:
import time

In [20]:
# Getting the average runtime (performance) on the model when it predicts 200 reviews
# 200 reviews is based on 


arr = []
for x in range(10):
    arr.append(tokenized_datasets_labelled_test.shuffle()[0:200]['TEXT'])
start = time.time()
for x in range(10):
    predictions = pipe.predict(arr[x])
stop = time.time()
avg = stop - start
print(avg/10)

18.783518576622008
